## Why should we test code?

Testing your code is a very important step in the release cycle.

Why is it important?
1. Until you execute a line of code, you don't know if that line can work at all.
1. Until you execute your code with a representative set of basic use cases, you don't know if the code can work end-to-end.
1. If you and possibly other people are going to modify your code, it's very easy to break it in unexpected ways. A suite of automated unit and integration tests gives you confidence you've not broken anything significant.
1. Tests can be used for profiling, to help you understand changes in your system's performance, and raise a flag if something degrades significantly.

## Types of testing

For any software application, both unit testing, as well as integration testing, is very important as each of them employs a unique process to test a software application.

**Unit testing** means testing individual modules of an application in isolation (without any interaction with dependencies) to confirm that those pieces of code are doing things right. **Integration testing** means checking if different modules are working fine when combined together as a group

There are all kinds of tests besides integration and unit tests, and they're all important.

* Regression tests, to make sure no bugs have recurred.
* Performance tests against various workloads, to characterize and guide improvement of performance.
* Stress tests to make sure the software can handle high loads, and work on a busy system.
* Resource tests, both to ensure resource consumption isn't unreasonable, and to ensure the code works as expected under low resource conditions.
* Change testing, for example when the IP address changes, the time is changed on the box, and so on.

## 1. Testing frameworks: `unittest`

`unittest` is a Python Standard Librabry module which provides a rich set of tools for constructing and running tests.

To achieve this, `unittest` supports some important concepts in an object-oriented way:

* **test fixture**
A test fixture represents the preparation needed to perform one or more tests, and any associate cleanup actions. This may involve, for example, creating temporary or proxy databases, directories, or starting a server process.

* **test case**
A test case is the individual unit of testing. It checks for a specific response to a particular set of inputs. unittest provides a base class, TestCase, which may be used to create new test cases.

* **test suite**
A test suite is a collection of test cases, test suites, or both. It is used to aggregate tests that should be executed together.

* **test runner**
A test runner is a component which orchestrates the execution of tests and provides the outcome to the user. The runner may use a graphical interface, a textual interface, or return a special value to indicate the results of executing the tests.


The building block of a test is the test case. With `unittest`, you can create a test case by subclassing the `unittest.TestCase` class. Each method starting with `test` will be an individual testing scenario. Test methods should contain at least one `assert*` method call as this is the essential role of a test: comparing actual results agaist expected results.

In [1]:
import unittest


class TestStringMethods(unittest.TestCase):

    def test_upper(self):
        self.assertEqual('foo'.upper(), 'FOO')

    def test_isupper(self):
        self.assertTrue('FOO'.isupper())
        self.assertFalse('Foo'.isupper())

    def test_split(self):
        s = 'hello world'
        self.assertEqual(s.split(), ['hello', 'world'])
        # check that s.split fails when the separator is not a string
        with self.assertRaises(TypeError):
            s.split(2)


unittest.main(argv=['-k', 'TestStringMethods'], verbosity=2, exit=False)

test_isupper (__main__.TestStringMethods.test_isupper) ... ok
test_split (__main__.TestStringMethods.test_split) ... ok
test_upper (__main__.TestStringMethods.test_upper) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.006s

OK


The following table lists the most commonly used assert methods:


| Method                                  | Checks that                       |
| --------------------------------------- | --------------------------------- |
| `assertEqual(a, b)`                     | `a == b`                          |
| `assertNotEqual(a, b)`                  | `a != b`                          |
| `assertTrue(x)`                         | `bool(x) is True`                 |
| `assertFalse(x)`                        | `bool(x) is False`                |
| `assertIs(a, b)`                        | `a is b`                          |
| `assertIsNot(a, b)`                     | `a is not b`                      |
| `assertIsNone(x)`                       | `x is None`                       |
| `assertIsNotNone(x)`                    | `x is not None`                   |
| `assertIn(a, b)`                        | `a in b`                          |
| `assertNotIn(a, b)`                     | `a not in b`                      |
| `assertIsInstance(a, b)`                | `isinstance(a, b)`                |
| `assertNotIsInstance(a, b)`             | `not isinstance(a, b)`            |
| `assertRaises(exc, fun, *args, **kwds)` | `fun(*args, **kwds)` raises `exc` |


Test fixtures represent the initial set-up needed before each test method or
before all the tests in a test case. This can be achieved by using the special
`setUp` method that will be called before every test run. Similarly, we can 
provide a `tearDown()` method that tidies up after the test method has been
run.


```python
import unittest

class WidgetTestCase(unittest.TestCase):
    def setUp(self):
        self.widget = Widget('The widget')

    def test_default_widget_size(self):
        self.assertEqual(self.widget.size(), (50,50),
                         'incorrect default size')

    def test_widget_resize(self):
        self.widget.resize(100,150)
        self.assertEqual(self.widget.size(), (100,150),
                         'wrong size after resize')

    def tearDown(self):
        self.widget.dispose()
```

### Command-Line Interface

The unittest module can be used from the command line to run tests from modules, classes or even individual test methods:

```shell
python -m unittest test_module1 test_module2
python -m unittest test_module.TestClass
python -m unittest test_module.TestClass.test_method
```

If you with to stop test run on the first error or failure, run the tests with
`-f, --failfast` command-line option:

```shell
python -m unittest -f
python -m unittest --failfast
```

For a list of all the command-line options:

```shell
python -m unittest -h
```

### Test Suites

It is recommended that you use `TestCase` implementations to group tests together according to the features they test. `unittest` provides another mechanism for grouping tests: **the test suite**, represented by `unittest`’s `TestSuite` class. In most cases, calling unittest.main() will do the right thing and collect all the module’s test cases for you and execute them.

However, should you want to customize the building of your test suite, you can do it yourself:

```python
def suite():
    suite = unittest.TestSuite()
    suite.addTest(WidgetTestCase('test_default_widget_size'))
    suite.addTest(WidgetTestCase('test_widget_resize'))
    return suite

if __name__ == '__main__':
    runner = unittest.TextTestRunner()
    runner.run(suite())
```

### Subtests

When there are very small differences among your tests, for instance some parameters, unittest allows you to distinguish them inside the body of a test method using the `subTest()` context manager.


In [2]:
import unittest


class NumbersTest(unittest.TestCase):

    def test_even(self):
        """
        Test that numbers between 0 and 5 are all even.
        """
        for i in range(0, 6):
            with self.subTest(f"Test failed for i = {i}"):
                self.assertEqual(i % 2, 0)
 

unittest.main(argv=['-k', 'NumbersTest'], verbosity=2, exit=False)

test_even (__main__.NumbersTest.test_even)
Test that numbers between 0 and 5 are all even. ... 
  test_even (__main__.NumbersTest.test_even) [Test failed for i = 1]
Test that numbers between 0 and 5 are all even. ... FAIL
  test_even (__main__.NumbersTest.test_even) [Test failed for i = 3]
Test that numbers between 0 and 5 are all even. ... FAIL
  test_even (__main__.NumbersTest.test_even) [Test failed for i = 5]
Test that numbers between 0 and 5 are all even. ... FAIL

FAIL: test_even (__main__.NumbersTest.test_even) [Test failed for i = 1]
Test that numbers between 0 and 5 are all even.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/s2/_752pzb50tg00mv9ggtr6p8c0000gn/T/ipykernel_46693/1847049937.py", line 12, in test_even
    self.assertEqual(i % 2, 0)
AssertionError: 1 != 0

FAIL: test_even (__main__.NumbersTest.test_even) [Test failed for i = 3]
Test that numbers between 0 and 5 are all even.
-----------

### Skipping tests and expected failures

Unittest supports skipping individual test methods and even whole classes of tests. In addition, it supports marking a test as an “expected failure,” a test that is broken and will fail, but shouldn’t be counted as a failure on a TestResult.

In [3]:
import sys
import unittest

VERSION = (1, 2)

def external_resource_available():
    return False


class MyTestCase(unittest.TestCase):

    @unittest.skip("demonstrating skipping")
    def test_nothing(self):
        self.fail("shouldn't happen")

    @unittest.skipIf(VERSION < (1, 3),
                     "not supported in this library version")
    def test_format(self):
        # Tests that work for only a certain version of the library.
        pass

    @unittest.skipUnless(sys.platform.startswith("win"), "requires Windows")
    def test_windows_support(self):
        # windows specific testing code
        pass

    def test_maybe_skipped(self):
        if not external_resource_available():
            self.skipTest("external resource not available")
        # test code that depends on the external resource
        pass
    
    @unittest.expectedFailure
    def test_fail(self):
        self.assertEqual(1, 0, "broken")
    
    
unittest.main(argv=['-k', 'MyTestCase'], verbosity=2, exit=False)

test_fail (__main__.MyTestCase.test_fail) ... expected failure
test_format (__main__.MyTestCase.test_format) ... skipped 'not supported in this library version'
test_maybe_skipped (__main__.MyTestCase.test_maybe_skipped) ... skipped 'external resource not available'
test_nothing (__main__.MyTestCase.test_nothing) ... skipped 'demonstrating skipping'
test_windows_support (__main__.MyTestCase.test_windows_support) ... skipped 'requires Windows'

----------------------------------------------------------------------
Ran 5 tests in 0.003s

OK (skipped=4, expected failures=1)


### Exercises 1

1. Write tests for `Employee.raise_salary()` using `unittest`. Consider all significant cases (raise with valid/invalid percent). Use subtests to test all valid values. Use a fixture for the `Employee` object.

## 2. Testing frameworks: `pytest`

`pytest` is a 3rd party library built as an alternative to Standard Library's `unittest`.

`pytest` supports execution of unittest test cases, but the real advantage of `pytest` comes by writing `pytest` test cases. `pytest` test cases are a series of functions in a Python file starting with the name `test_`.

`pytest` has some other great features:

* Tests are expressive and readable — no boilerplate code required
* Support for the built-in `assert` statement instead of using special `self.assert*()` methods
* Support for filtering for test cases
* Ability to rerun from the last failing test
* Marks and parametrized tests
* Modular fixtures
* An ecosystem of hundreds of plugins to extend the functionality

### Installation

Because it is a 3rd party library, you should install it using `pip`:

```
pip install pytest
```

### Writing a simple test

Tests in `pytest` are simple functions. Assertions are done using `assert` statement.

In order to execute pytests inside Jupyter Notebook, we're going to use a package called `ipytest`.

In [4]:
import sys                                                                                                                                                                                                  
!{sys.executable} -m pip install pytest
!{sys.executable} -m pip install ipytest

import ipytest
ipytest.autoconfig()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.9/339.9 kB 5.4 MB/s eta 0:00:00a 0:00:01
Using cached iniconfig-2.0.0-py3-none-any.whl (5.9 kB)


In [5]:
%%ipytest -qq


def func(x):
    return x + 1


def test_func_pass():
    assert func(3) == 4
    
def test_func_fail():
    assert func(3) == 5

Fxssss.F                                                                                     [100%]
============================================= FAILURES =============================================
______________________________________ NumbersTest.test_even _______________________________________

self = <__main__.NumbersTest testMethod=test_even>

    def test_even(self):
        """
        Test that numbers between 0 and 5 are all even.
        """
        for i in range(0, 6):
            with self.subTest(f"Test failed for i = {i}"):
>               self.assertEqual(i % 2, 0)
E               AssertionError: 1 != 0

/var/folders/s2/_752pzb50tg00mv9ggtr6p8c0000gn/T/ipykernel_46693/1847049937.py:12: AssertionError
__________________________________________ test_func_fail __________________________________________

    def test_func_fail():
>       assert func(3) == 5
E       assert 4 == 5
E        +  where 4 = func(3)

/var/folders/s2/_752pzb50tg00mv9ggtr6p8c0000gn/T/ipykernel_46

### Assert that an exception is raised

`pytest` implements a helper function that can be used with the `with` statement:

In [6]:
%%ipytest -qq

import pytest


def func():
    raise ValueError


def test_raises():
    with pytest.raises(ValueError):
        func()

Fxssss.                                                                                      [100%]
============================================= FAILURES =============================================
______________________________________ NumbersTest.test_even _______________________________________

self = <__main__.NumbersTest testMethod=test_even>

    def test_even(self):
        """
        Test that numbers between 0 and 5 are all even.
        """
        for i in range(0, 6):
            with self.subTest(f"Test failed for i = {i}"):
>               self.assertEqual(i % 2, 0)
E               AssertionError: 1 != 0

/var/folders/s2/_752pzb50tg00mv9ggtr6p8c0000gn/T/ipykernel_46693/1847049937.py:12: AssertionError
===================================== short test summary info ======================================
FAILED t_7497a9e926e34c6fa9f6fd9a453c1b90.py::NumbersTest::test_even - AssertionError: 1 != 0


## Command-line interface

`pytest`'s command line interface is more powerful than `unittest`s.

Running `pytest --help` will give you a list of all the arguments throught which the behaviour of the testing framework can be adjusted.

In [7]:
!{sys.executable} -m pytest --help



usage: __main__.py [options] [file_or_dir] [file_or_dir] [...]

positional arguments:
  file_or_dir

general:
  -k EXPRESSION         Only run tests which match the given substring
                        expression. An expression is a Python evaluable
                        expression where all names are substring-matched against
                        test names and their parent classes. Example: -k
                        'test_method or test_other' matches all test functions
                        and classes whose name contains 'test_method' or
                        'test_other', while -k 'not test_method' matches those
                        that don't contain 'test_method' in their names. -k 'not
                        test_method and not test_other' will eliminate the
                        matches. Additionally keywords are matched to classes
                        and functions containing extra names in their
                        'extra_keyword_matches' set, as we

### Invocation

You can invoke testing through the Python interpreter from the command line:

```
python -m pytest [...]
```

This is almost equivalent to invoking the command line script `pytest [...]` directly, except that calling via python will also add the current directory to `sys.path`.

### Running specific tests

Pytest supports several ways to run and select tests from the command-line.

Run tests in a module
```
pytest test_mod.py
```
Run tests in a directory
```
pytest testing/
```
Run tests by keyword expressions
```
pytest -k "MyClass and not method"
```
This will run tests which contain names that match the given string expression (case-insensitive), which can include Python operators that use filenames, class names and function names as variables. The example above will run `TestMyClass.test_something` but not `TestMyClass.test_method_simple`.

#### Run tests by node ids

Each collected test is assigned a unique nodeid which consist of the module filename followed by specifiers like class names, function names and parameters from parametrization, separated by :: characters.

To run a specific test within a module:

```
pytest test_mod.py::test_func
```
Another example specifying a test method in the command line:
```
pytest test_mod.py::TestClass::test_method
```

### Stopping on failures

To stop the testing process after the first (N) failures:

```
pytest -x           # stop after first failure
pytest --maxfail=2  # stop after two failures
```

Read more about `pytest`'s command-line interface [here](https://docs.pytest.org/en/6.2.x/usage.html).

## Fixtures

`pytest` fixtures offer dramatic improvements over the classic xUnit style of setup/teardown functions:

* fixtures have explicit names and are activated by declaring their use from test functions, modules, classes or whole projects.
* fixtures are implemented in a modular manner, as each fixture name triggers a fixture function which can itself use other fixtures.
* fixture management scales from simple unit to complex functional testing, allowing to parametrize fixtures and tests according to configuration and component options, or to re-use fixtures across function, class, module or whole test session scopes.
* teardown logic can be easily, and safely managed, no matter how many fixtures are used, without the need to carefully handle errors by hand or micromanage the order that cleanup steps are added.

We can tell pytest that a particular function is a fixture by decorating it with `@pytest.fixture`.

In [8]:
%%ipytest -qq


import pytest


class Fruit:
    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        return self.name == other.name


@pytest.fixture
def my_fruit():
    return Fruit("apple")


@pytest.fixture
def fruit_basket(my_fruit):
    return [Fruit("banana"), my_fruit]


def test_my_fruit_in_basket(my_fruit, fruit_basket):
    assert my_fruit in fruit_basket

Fxssss.                                                                                      [100%]
============================================= FAILURES =============================================
______________________________________ NumbersTest.test_even _______________________________________

self = <__main__.NumbersTest testMethod=test_even>

    def test_even(self):
        """
        Test that numbers between 0 and 5 are all even.
        """
        for i in range(0, 6):
            with self.subTest(f"Test failed for i = {i}"):
>               self.assertEqual(i % 2, 0)
E               AssertionError: 1 != 0

/var/folders/s2/_752pzb50tg00mv9ggtr6p8c0000gn/T/ipykernel_46693/1847049937.py:12: AssertionError
===================================== short test summary info ======================================
FAILED t_7497a9e926e34c6fa9f6fd9a453c1b90.py::NumbersTest::test_even - AssertionError: 1 != 0


## Marks

By using the `pytest.mark` helper you can easily set metadata on your test functions. Markers can be built-in or user-defined. You can list all the markers, including built-in and custom, using the CLI `pytest --markers`.

Here are some of the builtin markers:

* `usefixtures` - use fixtures on a test function or class
* `filterwarnings` - filter certain warnings of a test function
* `skip` - always skip a test function
* `skipif` - skip a test function if a certain condition is met
* `xfail` - produce an “expected failure” outcome if a certain condition is met
* `parametrize` - perform multiple calls to the same test function.

It’s easy to create custom markers or to apply markers to whole test classes or modules. Those markers can be used by plugins, and also are commonly used to select tests on the command-line with the -m option.

### Registering marks

You can register custom marks in your `pytest.ini` file like this:

```ini
[pytest]
markers =
    slow: marks tests as slow (deselect with '-m "not slow"')
    serial
```

### Marking tests

In [9]:
%%ipytest -qq -m slow


import pytest


def pytest_configure(config):
    config.addinivalue_line(
        "markers", "slow: marks tests as slow"
    )

def func():
    pass


@pytest.mark.slow
def test_func():
    assert func() is None

def test_serial():
    assert 1 == 0

.                                                                                            [100%]


## Parametrized tests

The builtin `pytest.mark.parametrize` decorator enables parametrization of arguments for a test function. Here is a typical example of a test function that implements checking that a certain input leads to an expected output:

In [10]:
%%ipytest -qq


import pytest


@pytest.mark.parametrize("test_input,expected", [("3+5", 8), ("2+4", 6), ("6*9", 42)])
def test_eval(test_input, expected):
    assert eval(test_input) == expected

Fxssss..F                                                                                    [100%]
============================================= FAILURES =============================================
______________________________________ NumbersTest.test_even _______________________________________

self = <__main__.NumbersTest testMethod=test_even>

    def test_even(self):
        """
        Test that numbers between 0 and 5 are all even.
        """
        for i in range(0, 6):
            with self.subTest(f"Test failed for i = {i}"):
>               self.assertEqual(i % 2, 0)
E               AssertionError: 1 != 0

/var/folders/s2/_752pzb50tg00mv9ggtr6p8c0000gn/T/ipykernel_46693/1847049937.py:12: AssertionError
________________________________________ test_eval[6*9-42] _________________________________________

test_input = '6*9', expected = 42

    @pytest.mark.parametrize("test_input,expected", [("3+5", 8), ("2+4", 6), ("6*9", 42)])
    def test_eval(test_input, expected):
>

### Exercises 2

1. Run all existing tests with `pytest`.
2. Create `search_term` tests using `pytest`. 